## **Statistical inference on crime statistics of Vancouver**

### **Introduction**

- Main Question *What are random variables of interest, spread across two or more categories that are interesting to compare*
- Why is this question important
- atleast 2 scientific publications acknowledging this
- Which data set will be used to answer the question

### **Preliminary results**

- Read the data set from the web 
- clean the data
- Plot some graphs answering some things about the question
- Compute estimates of the parameter you identified across your groups. Present this in a table. If relevant, include these estimates in your plot.

### **Methods: plan**

- In what ways is this report trustworthy
- Analyse the plot and explain what it means
- Explain why just plots are not enough to give a *stakeholder* [not sure what it means]
- What should you provide to address the gap
- What should you expect your results to be
- State the ***null hypothesis*** and the ***alternative hypothesis***
- What impact could such findings have?
- What future questions could this lead to?

### **References**

References on:
- Data base
- information about crime in vancouver
- what crime statistics are valuable